In [1]:
from pso import PSO, PSOConfig
from addict import Dict
from copy import deepcopy
import csv
import datetime
import numpy as np
import argparse
from utils import load_dataset, real_dataset_names
from sklearn.mixture import GaussianMixture

In [18]:

config_name = 'configs/params_landsat.json'

config = PSOConfig.from_json(config_name)

init_scales = [0.001, 0.003, 0.01, 0.03]

init_scales = [0.01]

n_runs = 3

results_list = []

dataset_name = 'landsat'
data = load_dataset(dataset_name)

results = {scale: {} for scale in init_scales}
for scale in init_scales:
    config.init_scale = scale

    for run in range(n_runs):
        results_list.append(pso_vs_em_experiment(config, data))

    for key in results_list[0].keys():
        results[scale][key] = f'{np.mean([item[key] for item in results_list])} +- {np.std([item[key] for item in results_list])}'


Basic precision matrix mean value 49.204366622925455
Random scattering results: [0]
Basic EM score 75.09418854904652
Iter 0 Particles reinit
PSO: Rerun EM best fintess score 71.08001832969492
PSO: Max fitness score 60.043971841724634
Iter 5 Particles reinit
PSO: Rerun EM best fintess score 71.92561470658876
PSO: Max fitness score 61.7663145271997
Iter 10 Particles reinit
PSO: Rerun EM best fintess score 72.31233938466802
PSO: Max fitness score 62.87517801760006
Iter 15 Particles reinit
PSO: Rerun EM best fintess score 73.5028967943923
PSO: Max fitness score 64.58953611399673
Iter 20 Particles reinit
PSO: Rerun EM best fintess score 74.0354057728977
PSO: Max fitness score 66.41415833603577
Reference GMM 2 * n_particles inits, T1 * T2 iters: 75.33419335887442
Basic precision matrix mean value 57.485524885474916
Random scattering results: [0]
Basic EM score 75.88210681227058
Iter 0 Particles reinit
PSO: Rerun EM best fintess score 71.5002964262168
PSO: Max fitness score 62.471447095869344

In [17]:
results

{0.003: {'em': '75.78490957565921 +- 0.16608005746356136',
  'pso': '75.88803917437002 +- 0.14001455099619786',
  'ref_gmm_score': '75.74072038078435 +- 0.1933872221674909',
  'random_scattering': '0.0 +- 0.0'}}

In [3]:

def pso_vs_em_experiment(config, data):

    pso = PSO(data, config)
    n_iters = config.n_iters
    T1 = config.T1

    print(f'Basic precision matrix mean value {np.sqrt(np.mean(pso.basic_prec_matr))}')

    random_scattering_scores = [0]

    for i in range(0):
        pso.reinit_particles()
        rerun_em_fintess_score_list = pso.run_em_particles_fintess_score()
        random_scattering_scores.append(max(rerun_em_fintess_score_list))

    print(f'Random scattering results: {random_scattering_scores}')

    pso.reinit_particles()

    gmm_reinit_pso_scores = []
    # M * T_1 * T_2 EM iterations for basic EM init
    print('Basic EM score', pso.basic_gmm_score)

    # M * T_1 * T_2 EM iterations for EM init from PSO particles positions
    for i in range(n_iters + 1):
        pso.step()
        if  i % (n_iters // (T1 - 1)) == 0:
            print(f'Iter {i} Particles reinit')

            rerun_em_fintess_score_list = pso.run_em_particles_fintess_score()
            fintess_score_list = pso.get_particles_fitness_scores()

            gmm_reinit_pso_scores.append(max(rerun_em_fintess_score_list))
            print('PSO: Rerun EM best fintess score', max(rerun_em_fintess_score_list))
            print(f'PSO: Max fitness score {max(fintess_score_list)}')

    init_gmm_em_score = pso.basic_gmm_score

    # init big EM with 2 * M * T_1 * T_2 EM iteration budget
    
    T1 = config.T1
    T2 = config.T2
    
    reference_gmm = GaussianMixture(n_components=config.n_components, covariance_type='full', n_init=2 * config.n_particles, max_iter=T2 * T1,  init_params=config.EM_init_method)
    reference_gmm.fit(data)

    print('Reference GMM 2 * n_particles inits, T1 * T2 iters:', reference_gmm.score(data))

    return {'em': init_gmm_em_score, 'pso': max(gmm_reinit_pso_scores), 'ref_gmm_score': reference_gmm.score(data), 'random_scattering': max(random_scattering_scores)}
    